In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:44041")
client

Client Scheduler: tcp://127.0.0.1:44041 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 14 Cores: 56 Memory: 236.55 GB


In [2]:
import numpy as np
import xarray as xr
from xmitgcm import open_mdsdataset
from xgcm.grid import Grid
import os.path as op
from fastjmd95 import rho as densjmd95
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ddir = '/tank/chaocean/'
savedir = '/tank/topog/tuchida/TWA/'

In [4]:
grav = 9.81
rho0 = 999.8
yearfirst = 1974
yearlast = 1995
# ph is made from averaged data so firstfistit should be 790560
firstfirstit = 790560
yearyearfirst = 1963
# firstfirstit is the first iteration count in year 1970;
dnf = 2160
nfile = 73
ychunk = 450
xchunk = 250

ystart = -15
yend = 50

In [5]:
def sound_speed(s,th,p):
    """
    s            : salinity (psu)
    th           : potential temperature (deg C, ITS-90)
    p            : pressure (dbar)
    """
    th2 = th**2
    sqrts = np.sqrt(s)

    anum = ( 9.99843699e2 +
            th *( 7.35212840 +
                 th *(-5.45928211e-2 +
                      th * 3.98476704e-4
                     )
                ) +
            s *( 2.96938239 -
                th * 7.23268813e-3 +
                s * 2.12382341e-3
               )
           )
    aden = (1. +
            th *( 7.28606739e-3 +
                 th *(-4.60835542e-5 +
                      th *( 3.68390573e-7 +
                           th * 1.80809186e-10
                          )
                     )
                ) +
            s *( 2.14691708e-3 -
                th *( 9.27062484e-6 +
                     th2 * 1.78343643e-10
                    ) +
                sqrts *( 4.76534122e-6 +
                        th2 * 1.63410736e-9
                       )
               )
           )
    

    anum_p = ( 1.04004591e-2 +
              th2 * 1.03970529e-7 +
              s * 5.18761880e-6
             )
    aden_p = 5.30848875e-6 * xr.ones_like(s)


    pth = p*th
    anum = anum + p *( 1.04004591e-2 +
                      th2 *  1.03970529e-7 +
                      s *  5.18761880e-6 -
                      p *( 3.24041825e-8 +
                          th2 *  1.23869360e-11
                         )
                     )
    aden = aden + p *( 5.30848875e-6 -
                      pth *(th2 *  3.03175128e-16 +
                            p *  1.27934137e-17
                           )
                     )


    anum_p = anum_p - p *( 6.4808365e-8 + th2 * 2.4773872e-11)

    aden_p = aden_p - pth *(th2 * 6.06350256e-16 +
                            p * 3.83802411e-17)


    rec_aden = aden**-1

    rho = anum*rec_aden
    rho_p = (anum_p - aden_p*rho)*rec_aden
    
    return np.sqrt(rho_p*1e-4)**-1

In [13]:
years = np.arange(2008,2013,dtype=int)
ntimes = np.arange(7886160,8672400+dnf,dnf).reshape(len(years),nfile)
ntimes[1,60]

8173440

In [14]:
yystart = 1
yy = yystart

for year in years[yystart:]:
    if year == years[yystart]:
        mtimes = ntimes[yy,62:]
    else:
        mtimes = ntimes[yy]
    for itime in mtimes:
        for nmemb in range(24):
            if nmemb == 0:
                ds = open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' % (nmemb,year)), 
                                     grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
                                     prefix=['diag_ocnTave','diag_ocnSurf'], 
                                     delta_t=2e2, iters=itime,
                                     chunks={'XC':xchunk,'XG':xchunk,'YC':ychunk,'YG':ychunk}
                                    ).sel(YC=slice(ystart-12**-1,yend+2*12**-1),
                                          YG=slice(ystart-12**-1,yend+2*12**-1))
            else:
                ds = xr.concat([ds, open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                                            % (nmemb,year)), 
                                                    grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
                                                    prefix=['diag_ocnTave','diag_ocnSurf'], 
                                                    delta_t=2e2, iters=itime,
                                                    chunks={'XC':xchunk,'XG':xchunk,'YC':ychunk,'YG':ychunk}
                                                   ).sel(YC=slice(ystart-12**-1,yend+2*12**-1),
                                                         YG=slice(ystart-12**-1,yend+2*12**-1))
                               ], 'nmemb')
        ds.coords['nmemb'] = ('nmemb',range(24))
            
        if itime == mtimes[0]:
            pres = rho0*grav*(-ds.Z) * 1e-4  # convert to [dbar]
            grid = Grid(ds, periodic=['X'])
            
        theta = ds.THETA.where(ds.maskC!=0.)
        salt = ds.SALT.where(ds.maskC!=0.)
        eta = ds.ETAN.where(ds.maskInC!=0.)
            
        rho = xr.apply_ufunc(densjmd95, salt, theta, pres,             # pressure in [dbar]!!!!
                             dask='parallelized', output_dtypes=[float,]
                            ).where(ds.maskC!=0.)
        rho = -grav/rho0*(rho-rho0).compute()
            
#             rhoa=(rho[:,:,:-1]+rho[:,:,1:])/2; rhoa[:,:,-1]=rhoa[:,:,-2];
        rhoa = grid.interp(rho,'Z',boundary='fill',to='outer').isel(Zp1=slice(1,None))
        phiHC = xr.ones_like(rho) * np.nan
        for k in range(len(ds.Z)):
            if k == 0:
                phiHC.isel(Z=0)[:] = grav*eta - (ds.drC.isel(Zp1=0).data*rho.isel(Z=0))
            else:
                phiHC.isel(Z=k)[:] = phiHC.isel(Z=k-1) - (ds.drC.isel(Zp1=k).data*rhoa.isel(Zp1=k-1).data)
        
        del rho, rhoa
        
        Cs = sound_speed(salt, theta, pres).compute()
                    
        dsave = phiHC.to_dataset(name='PHIHYD')
        dsave['cs'] = Cs
        dsave.to_zarr(op.join(savedir,'PHI_5Dave/run%4d/%010d/' 
                              % (year,itime)), mode='w')
        
        del Cs, phiHC
        ds.close()
        dsave.close()
    
        print(itime)
#     print(year)
    yy += 1

8177760
8179920
8182080
8184240
8186400
8188560
8190720
8192880
8195040
8197200
8199360
8201520
8203680
8205840
8208000
8210160
8212320
8214480
8216640
8218800
8220960
8223120
8225280
8227440
8229600
8231760
8233920
8236080
8238240
8240400
8242560
8244720
8246880
8249040
8251200
8253360
8255520
8257680
8259840
8262000
8264160
8266320
8268480
8270640
8272800
8274960
8277120
8279280
8281440
8283600
8285760
8287920
8290080
8292240
8294400
8296560
8298720
8300880
8303040
8305200
8307360
8309520
8311680
8313840
8316000
8318160
8320320
8322480
8324640
8326800
8328960
8331120
8333280
8335440
8337600
8339760
8341920
8344080
8346240
8348400
8350560
8352720
8354880
8357040
8359200
8361360
8363520
8365680
8367840
8370000
8372160
8374320
8376480
8378640
8380800
8382960
8385120
8387280
8389440
8391600
8393760
8395920
8398080
8400240
8402400
8404560
8406720
8408880
8411040
8413200
8415360
8417520
8419680
8421840
8424000
8426160
8428320
8430480
8432640
8434800
8436960
8439120
8441280
8443440
8445600


In [18]:
rhoa = grid.interp(rho,'Z',boundary='fill',to='outer').isel(Zp1=slice(1,None))
rhoa.isel(nmemb=0,time=0,Zp1=0)

<xarray.DataArray (YC: 783, XC: 1000)>
array([[-0.24382809, -0.24366633, -0.24352696, ..., -0.24456701,
        -0.24430209, -0.24404149],
       [-0.24387073, -0.24369602, -0.24353855, ..., -0.2445686 ,
        -0.24432638, -0.24408166],
       [-0.2439523 , -0.24375732, -0.24357305, ..., -0.24461631,
        -0.24440115, -0.24416916],
       ...,
       [-0.25713723, -0.25729663, -0.25745532, ..., -0.2568069 ,
        -0.25688279, -0.25699493],
       [-0.257087  , -0.25723612, -0.25739633, ..., -0.25682468,
        -0.25687481, -0.25696322],
       [-0.25704538, -0.25717604, -0.25732836, ..., -0.25685906,
        -0.25688361, -0.25694618]])
Coordinates:
    nmemb    int64 0
    time     float64 1.577e+09
    Zp1      >f4 -6.0935454
  * YC       (YC) >f4 -15.0833 -14.999967 -14.916634 ... 50.000034 50.08337
  * XC       (XC) >f4 262.00003 262.0861 262.17215 ... 347.88644 347.9725

In [19]:
np.testing.assert_array_equal(rhoa.isel(nmemb=0,time=0,Zp1=0).data,
                              grid.interp(rho,'Z',boundary='fill').isel(nmemb=0,time=0,Zl=1).data)

In [16]:
grid.interp(rho,'Z',boundary='fill').isel(nmemb=0,time=0,Zl=1)

<xarray.DataArray (YC: 783, XC: 1000)>
array([[-0.24382809, -0.24366633, -0.24352696, ..., -0.24456701,
        -0.24430209, -0.24404149],
       [-0.24387073, -0.24369602, -0.24353855, ..., -0.2445686 ,
        -0.24432638, -0.24408166],
       [-0.2439523 , -0.24375732, -0.24357305, ..., -0.24461631,
        -0.24440115, -0.24416916],
       ...,
       [-0.25713723, -0.25729663, -0.25745532, ..., -0.2568069 ,
        -0.25688279, -0.25699493],
       [-0.257087  , -0.25723612, -0.25739633, ..., -0.25682468,
        -0.25687481, -0.25696322],
       [-0.25704538, -0.25717604, -0.25732836, ..., -0.25685906,
        -0.25688361, -0.25694618]])
Coordinates:
    nmemb    int64 0
    time     float64 1.577e+09
    Zl       >f4 -6.0935454
  * YC       (YC) >f4 -15.0833 -14.999967 -14.916634 ... 50.000034 50.08337
  * XC       (XC) >f4 262.00003 262.0861 262.17215 ... 347.88644 347.9725

In [15]:
.5*(rho.isel(nmemb=0,time=0,Z=0)+rho.isel(nmemb=0,time=0,Z=1).data)

<xarray.DataArray (YC: 783, XC: 1000)>
array([[-0.24382809, -0.24366633, -0.24352696, ..., -0.24456701,
        -0.24430209, -0.24404149],
       [-0.24387073, -0.24369602, -0.24353855, ..., -0.2445686 ,
        -0.24432638, -0.24408166],
       [-0.2439523 , -0.24375732, -0.24357305, ..., -0.24461631,
        -0.24440115, -0.24416916],
       ...,
       [-0.25713723, -0.25729663, -0.25745532, ..., -0.2568069 ,
        -0.25688279, -0.25699493],
       [-0.257087  , -0.25723612, -0.25739633, ..., -0.25682468,
        -0.25687481, -0.25696322],
       [-0.25704538, -0.25717604, -0.25732836, ..., -0.25685906,
        -0.25688361, -0.25694618]])
Coordinates:
    Depth    (YC, XC) >f4 3224.8105 3160.4026 3162.2163 ... 2158.6245 2124.6829
    hFacC    (YC, XC) >f4 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
    PHrefC   >f4 29.88884
    maskC    (YC, XC) bool True True True True True ... True True True True True
    iter     int64 7886160
    maskInC  (YC, XC) bool True True True True True ... True True True True True
    rA       (YC, XC) >f4 85589464.0 85589464.0 ... 56879988.0 56879988.0
    drF      >f4 6.0935454
  * YC       (YC) >f4 -15.0833 -14.999967 -14.916634 ... 50.000034 50.08337
    Z        >f4 -3.0467727
    time     float64 1.577e+09
  * XC       (XC) >f4 262.00003 262.0861 262.17215 ... 347.88644 347.9725
    nmemb    int64 0